# Financial Data Extraction

1. Sources

    * [Wikipedia]("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    * [iex Trading APIs]("https://iextrading.com/developer/")
    * [Yahoo Finance API]("https://rapidapi.com/apidojo/api/yahoo-finance1?")


2. Data Extraction

    2.1. Fortune 500 Companies Table
    
    2.2. Daily stocks information for Fortune 500 Companies
    
    2.3. Pre-selection of Financial Services Companies in Insurance Industry
    
        * Historical Stocks Data 
        
        * Daily news 

        * Last 4 Financial Statements 
            

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Financial-Data-Extraction" data-toc-modified-id="Financial-Data-Extraction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Financial Data Extraction</a></span></li><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dependencies</a></span></li><li><span><a href="#Scrape/Extract-Fortune-500-Table" data-toc-modified-id="Scrape/Extract-Fortune-500-Table-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Scrape/Extract Fortune 500 Table</a></span><ul class="toc-item"><li><span><a href="#Type-of-Data-&amp;-Convert-to-DataFrame" data-toc-modified-id="Type-of-Data-&amp;-Convert-to-DataFrame-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Type of Data &amp; Convert to DataFrame</a></span></li><li><span><a href="#Make-symbols-to-list" data-toc-modified-id="Make-symbols-to-list-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Make symbols to list</a></span></li></ul></li><li><span><a href="#Extract-Daily-Stocks-Data-for-fortune-500" data-toc-modified-id="Extract-Daily-Stocks-Data-for-fortune-500-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Extract Daily Stocks Data for fortune 500</a></span><ul class="toc-item"><li><span><a href="#Display-Data-Output" data-toc-modified-id="Display-Data-Output-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Display Data Output</a></span></li><li><span><a href="#Convert-data-to-Data-Frame-and-save-in-CSV-&amp;-JSON" data-toc-modified-id="Convert-data-to-Data-Frame-and-save-in-CSV-&amp;-JSON-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Convert data to Data Frame and save in CSV &amp; JSON</a></span></li></ul></li><li><span><a href="#Extract-Historical-Stocks-Data" data-toc-modified-id="Extract-Historical-Stocks-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Extract Historical Stocks Data</a></span><ul class="toc-item"><li><span><a href="#Filter-Financial-Insurances-Fortune-500-Companies" data-toc-modified-id="Filter-Financial-Insurances-Fortune-500-Companies-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Filter Financial Insurances Fortune 500 Companies</a></span></li><li><span><a href="#Obtain-symbols-&amp;-save-table" data-toc-modified-id="Obtain-symbols-&amp;-save-table-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Obtain symbols &amp; save table</a></span></li><li><span><a href="#Stocks-Price-&amp;-Dividend---Financial-Services---Insurance" data-toc-modified-id="Stocks-Price-&amp;-Dividend---Financial-Services---Insurance-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Stocks Price &amp; Dividend - Financial Services - Insurance</a></span></li><li><span><a href="#View-Output-Data-&amp;-Create-Data-Frame" data-toc-modified-id="View-Output-Data-&amp;-Create-Data-Frame-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>View Output Data &amp; Create Data Frame</a></span></li><li><span><a href="#Save-data-to-CSV-&amp;-JSON" data-toc-modified-id="Save-data-to-CSV-&amp;-JSON-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Save data to CSV &amp; JSON</a></span></li></ul></li><li><span><a href="#Extract-News-Stock-Data---Financial-Services---Insurance" data-toc-modified-id="Extract-News-Stock-Data---Financial-Services---Insurance-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Extract News Stock Data - Financial Services - Insurance</a></span><ul class="toc-item"><li><span><a href="#Daily-News-Stocks---Financial-Services---Insurance" data-toc-modified-id="Daily-News-Stocks---Financial-Services---Insurance-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Daily News Stocks - Financial Services - Insurance</a></span></li><li><span><a href="#Print-output-data-and-check-length-of-extracted-information" data-toc-modified-id="Print-output-data-and-check-length-of-extracted-information-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Print output data and check length of extracted information</a></span></li><li><span><a href="#Loop-to-obtain-only-the-key-data" data-toc-modified-id="Loop-to-obtain-only-the-key-data-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Loop to obtain only the key data</a></span></li><li><span><a href="#Looping,-append-and-create-a-Data-Frame" data-toc-modified-id="Looping,-append-and-create-a-Data-Frame-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Looping, append and create a Data Frame</a></span></li><li><span><a href="#Visualize-all-output-data" data-toc-modified-id="Visualize-all-output-data-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Visualize all output data</a></span></li><li><span><a href="#Save-data-to-CSV-&amp;-JSON" data-toc-modified-id="Save-data-to-CSV-&amp;-JSON-6.6"><span class="toc-item-num">6.6&nbsp;&nbsp;</span>Save data to CSV &amp; JSON</a></span></li></ul></li><li><span><a href="#Extracting-Financial-Statements" data-toc-modified-id="Extracting-Financial-Statements-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Extracting Financial Statements</a></span><ul class="toc-item"><li><span><a href="#Financial-Statements---Last-4-Quarters" data-toc-modified-id="Financial-Statements---Last-4-Quarters-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Financial Statements - Last 4 Quarters</a></span></li><li><span><a href="#Display-output-data" data-toc-modified-id="Display-output-data-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Display output data</a></span></li><li><span><a href="#Convert-dict-to-DataFrame-list" data-toc-modified-id="Convert-dict-to-DataFrame-list-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Convert dict to DataFrame list</a></span></li><li><span><a href="#Concatenante-Data-for-each-financial-Statements" data-toc-modified-id="Concatenante-Data-for-each-financial-Statements-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Concatenante Data for each financial Statements</a></span></li><li><span><a href="#Display-Data-Frame-Results" data-toc-modified-id="Display-Data-Frame-Results-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>Display Data Frame Results</a></span></li><li><span><a href="#Save-data-to-CSV" data-toc-modified-id="Save-data-to-CSV-7.6"><span class="toc-item-num">7.6&nbsp;&nbsp;</span>Save data to CSV</a></span></li></ul></li><li><span><a href="#Move-to-Transformation" data-toc-modified-id="Move-to-Transformation-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Move to Transformation</a></span></li></ul></div>

# Dependencies

In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv
from tqdm import tqdm_notebook as tqdm
from bs4 import BeautifulSoup as bs
from splinter import Browser


# Scrape/Extract Fortune 500 Table

In [42]:
# Use Pandas to scrape the table containing stocks data.
url_stock = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Convert table to html
stock_table = pd.read_html(url_stock)

# Print table 
print(stock_table)

[    Symbol                Security SEC filings             GICS Sector  \
0      MMM              3M Company     reports             Industrials   
1      ABT     Abbott Laboratories     reports             Health Care   
2     ABBV             AbbVie Inc.     reports             Health Care   
3     ABMD             ABIOMED Inc     reports             Health Care   
4      ACN           Accenture plc     reports  Information Technology   
..     ...                     ...         ...                     ...   
500    XYL              Xylem Inc.     reports             Industrials   
501    YUM         Yum! Brands Inc     reports  Consumer Discretionary   
502    ZBH  Zimmer Biomet Holdings     reports             Health Care   
503   ZION           Zions Bancorp     reports              Financials   
504    ZTS                  Zoetis     reports             Health Care   

                  GICS Sub Industry     Headquarters Location  \
0          Industrial Conglomerates       St.

## Type of Data & Convert to DataFrame

In [43]:
# Identify what type is the output data
type(stock_table)

# Ensure to select the first part of the list
stock_table_df = pd.DataFrame(stock_table[0])

# Save raw data to csv
stock_table_df.to_csv("../stocks_data/raw_fortune500table.csv", index=False, encoding='utf8')

stock_table_df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


## Make symbols to list

In [44]:
# Make he Symbol column to a list 
symbols_list = stock_table_df["Symbol"].values.tolist()
symbols_list


['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AMG',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'AGN',
 'ADS',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ARNC',
 'ANET',
 'AJG',
 'AIZ',
 'ATO',
 'T',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BHGE',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BBT',
 'BDX',
 'BRK.B',
 'BBY',
 'BIIB',
 'BLK',
 'HRB',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BF.B',
 'CHRW',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CPRI',
 'CAH',
 'KMX',
 'CCL',
 'CAT',
 'CBOE',
 'CBRE',
 'CBS',
 'CE',
 'CELG',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'XEC',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CM

# Extract Daily Stocks Data for fortune 500

In [45]:
base_url = "https://cloud.iexapis.com/stable/stock/"
key_token = "/quote?token=pk_e4c3fa15e49a4efca388fcfbcece0eac"

# Empty list to store data
stocks_data = []

# Start counter for the loop

symbol_counter = 1

#symbols_list2 = ['MMM','ABT','AIG','HAL','SLB'] # test list

#Print statement to inform when data acquisition start
print(f"Beginning Data Acquisition!!!")
print(f"------------&&&&-------------------")

for symbol in tqdm(symbols_list): #Note that we are using the symbol list
    try: 
        response = requests.get(base_url+symbol+key_token).json()
        stocks_data.append(response)
        
        print(f"Data Acquisition {symbol_counter} | Stock Symbol : {symbol}")
        symbol_counter = symbol_counter + 1
    except:
        print("Stock not found...!!")
    continue
    
# Print statement to inform when all records have been acquired.
print(f"------------&&&&---------------")
print(f"Data Acquisition Complete")
print(f"------------&&&&---------------")      


Beginning Data Acquisition!!!
------------&&&&-------------------


Data Acquisition 1 | Stock Symbol : MMM
Data Acquisition 2 | Stock Symbol : ABT
Data Acquisition 3 | Stock Symbol : ABBV
Data Acquisition 4 | Stock Symbol : ABMD
Data Acquisition 5 | Stock Symbol : ACN
Data Acquisition 6 | Stock Symbol : ATVI
Data Acquisition 7 | Stock Symbol : ADBE
Data Acquisition 8 | Stock Symbol : AMD
Data Acquisition 9 | Stock Symbol : AAP
Data Acquisition 10 | Stock Symbol : AES
Data Acquisition 11 | Stock Symbol : AMG
Data Acquisition 12 | Stock Symbol : AFL
Data Acquisition 13 | Stock Symbol : A
Data Acquisition 14 | Stock Symbol : APD
Data Acquisition 15 | Stock Symbol : AKAM
Data Acquisition 16 | Stock Symbol : ALK
Data Acquisition 17 | Stock Symbol : ALB
Data Acquisition 18 | Stock Symbol : ARE
Data Acquisition 19 | Stock Symbol : ALXN
Data Acquisition 20 | Stock Symbol : ALGN
Data Acquisition 21 | Stock Symbol : ALLE
Data Acquisition 22 | Stock Symbol : AGN
Data Acquisition 23 | Stock Symbol : ADS
Data Acquisition 24 | Stock Symbol : LNT
Data Acquisition 25

Data Acquisition 197 | Stock Symbol : FITB
Data Acquisition 198 | Stock Symbol : FE
Data Acquisition 199 | Stock Symbol : FRC
Data Acquisition 200 | Stock Symbol : FISV
Data Acquisition 201 | Stock Symbol : FLT
Data Acquisition 202 | Stock Symbol : FLIR
Data Acquisition 203 | Stock Symbol : FLS
Data Acquisition 204 | Stock Symbol : FMC
Data Acquisition 205 | Stock Symbol : F
Data Acquisition 206 | Stock Symbol : FTNT
Data Acquisition 207 | Stock Symbol : FTV
Data Acquisition 208 | Stock Symbol : FBHS
Data Acquisition 209 | Stock Symbol : FOXA
Data Acquisition 210 | Stock Symbol : FOX
Data Acquisition 211 | Stock Symbol : BEN
Data Acquisition 212 | Stock Symbol : FCX
Data Acquisition 213 | Stock Symbol : GPS
Data Acquisition 214 | Stock Symbol : GRMN
Data Acquisition 215 | Stock Symbol : IT
Data Acquisition 216 | Stock Symbol : GD
Data Acquisition 217 | Stock Symbol : GE
Data Acquisition 218 | Stock Symbol : GIS
Data Acquisition 219 | Stock Symbol : GM
Data Acquisition 220 | Stock Symbo

Data Acquisition 391 | Stock Symbol : PVH
Data Acquisition 392 | Stock Symbol : QRVO
Data Acquisition 393 | Stock Symbol : PWR
Data Acquisition 394 | Stock Symbol : QCOM
Data Acquisition 395 | Stock Symbol : DGX
Data Acquisition 396 | Stock Symbol : RL
Data Acquisition 397 | Stock Symbol : RJF
Data Acquisition 398 | Stock Symbol : RTN
Data Acquisition 399 | Stock Symbol : O
Data Acquisition 400 | Stock Symbol : REG
Data Acquisition 401 | Stock Symbol : REGN
Data Acquisition 402 | Stock Symbol : RF
Data Acquisition 403 | Stock Symbol : RSG
Data Acquisition 404 | Stock Symbol : RMD
Data Acquisition 405 | Stock Symbol : RHI
Data Acquisition 406 | Stock Symbol : ROK
Data Acquisition 407 | Stock Symbol : ROL
Data Acquisition 408 | Stock Symbol : ROP
Data Acquisition 409 | Stock Symbol : ROST
Data Acquisition 410 | Stock Symbol : RCL
Data Acquisition 411 | Stock Symbol : CRM
Data Acquisition 412 | Stock Symbol : SBAC
Data Acquisition 413 | Stock Symbol : SLB
Data Acquisition 414 | Stock Symb

## Display Data Output

In [46]:
# Show data extracted
stocks_data

[{'symbol': 'MMM',
  'companyName': '3M Co.',
  'primaryExchange': 'New York Stock Exchange',
  'calculationPrice': 'close',
  'open': 164.64,
  'openTime': 1567776600508,
  'close': 163.94,
  'closeTime': 1567800108143,
  'high': 164.88,
  'low': 162.95,
  'latestPrice': 163.94,
  'latestSource': 'Close',
  'latestTime': 'September 6, 2019',
  'latestUpdate': 1567800108143,
  'latestVolume': 1762315,
  'iexRealtimePrice': 163.93,
  'iexRealtimeSize': 14,
  'iexLastUpdated': 1567799990885,
  'delayedPrice': 163.94,
  'delayedPriceTime': 1567800600012,
  'extendedPrice': 163.94,
  'extendedChange': 0,
  'extendedChangePercent': 0,
  'extendedPriceTime': 1567810976991,
  'previousClose': 164.23,
  'previousVolume': 2421764,
  'change': -0.29,
  'changePercent': -0.00177,
  'volume': 1762315,
  'iexMarketPercent': 0.0394577586867274,
  'iexVolume': 69537,
  'avgTotalVolume': 2935039,
  'iexBidPrice': 0,
  'iexBidSize': 0,
  'iexAskPrice': 0,
  'iexAskSize': 0,
  'marketCap': 94311239260,


## Convert data to Data Frame and save in CSV & JSON

In [47]:
# Convert data extracted to data frame
stocks_data_df = pd.DataFrame(stocks_data)
stocks_data_df

# Save in csv and json file
stocks_data_df.to_csv("../stocks_data/raw_stock_data_fortune500.csv", index=False, encoding='utf8')

stocks_data_df.to_json("../stocks_data/raw_stock_data_fortune500.json", orient='columns')


# Extract Historical Stocks Data

## Filter Financial Insurances Fortune 500 Companies

In [37]:
fortune_df_finacials = stock_table_df[stock_table_df["GICS Sector"] == "Financials"]
fortune_df_finacials 

# Now select all the rows where the column GICS Sub Industry contains string Insura
# https://davidhamann.de/2017/06/26/pandas-select-elements-by-string/

fortune_df_finacials = fortune_df_finacials[fortune_df_finacials["GICS Sub Industry"].str.contains('Insura')]
fortune_df_finacials = fortune_df_finacials.reset_index()

# Drop index column
fortune_df_finacials.drop(columns = ['index'], inplace = True)

In [39]:
# Display data
fortune_df_finacials

,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,AFL,AFLAC Inc,reports,Financials,Life & Health Insurance,"Columbus, Georgia",1999-05-28,4977,1955
1,ALL,Allstate Corp,reports,Financials,Property & Casualty Insurance,"Northfield Township, Illinois",1995-07-13,899051,1931
2,AIG,American International Group,reports,Financials,Property & Casualty Insurance,"New York, New York",1980-03-31,5272,1919
3,AON,Aon plc,reports,Financials,Insurance Brokers,"London, United Kingdom",1996-04-23,315293,1982 (1919)
4,AJG,Arthur J. Gallagher & Co.,reports,Financials,Insurance Brokers,"Itasca, Illinois",2016-05-31,354190,NaN
5,AIZ,Assurant,reports,Financials,Multi-line Insurance,"New York, New York",2007-04-10,1267238,NaN
6,CB,Chubb Limited,reports,Financials,Property & Casualty Insurance,"Zurich, Switzerland",2010-07-15,896159,NaN
7,CINF,Cincinnati Financial,reports,Financials,Property & Casualty Insurance,"Fairfield, Ohio",1997-12-18,20286,NaN
8,GL,Globe Life Inc.,reports,Financials,Life & Health Insurance,"McKinney, Texas",1989-04-30,320335,NaN
9,HIG,Hartford Financial Svc.Gp.,reports,Financials,Property & Casualty Insurance,"Hartford, Connecticut",NaN,874766,NaN


## Obtain symbols & save table

In [40]:
# Convert symbols columns to a list
symbols_insurance = fortune_df_finacials['Symbol'].values.tolist()
symbols_insurance

['AFL',
 'ALL',
 'AIG',
 'AON',
 'AJG',
 'AIZ',
 'CB',
 'CINF',
 'GL',
 'HIG',
 'LNC',
 'L',
 'MMC',
 'MET',
 'PFG',
 'PGR',
 'PRU',
 'TRV',
 'UNM',
 'WLTW']

In [41]:
# Savings to CSV and JSON files

fortune_df_finacials.to_csv("../stocks_data/raw_20_fortune_finacials.csv", index=False, encoding='utf8')

fortune_df_finacials.to_json("../stocks_data/raw_20_fortune_finacials.json", orient='columns')

##  Stocks Price & Dividend - Financial Services - Insurance 

In [8]:
# Yahoo Finance API

historical_url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-historical-data"

headers = {
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
    'x-rapidapi-key': "09ec0160admsh2d5563a7feaa3efp1e2db5jsn5787845ed7fc"
    }

prices_list = []
event_list = []

stock_history_counter = 1

# List of 20 Stocks from Insurance industry

# Back up variable -- symbols_insurance = ['AIG','AFL','ALL','AON','AJG','CB','CINF','RE','GL','HIG','MMC','MET','PFG','PGR','PRU','TRV','UNM','WLTW','LNC','L']

# symbols_insurance = ['AIG','AFL','ALL'] # Test list

#Print statement to inform when data acquisition start
print(f"Beginning Data Acquisition!!!")
print(f"------------&&&&-------------------")

for s in tqdm(symbols_insurance):
    
    querystring = {"frequency":"1wk","filter":"history","period1":"1136219674","period2":"1567528474","symbol": s}

    try: 
        response_history = requests.get(historical_url,headers = headers, params=querystring).json()
        
        prices = response_history['prices']
        events = response_history['eventsData']
        
        for price in prices:
            price['symbol'] = s
            prices_list.append(price)
        
        for event in events:
            event['symbol'] = s
            event_list.append(event)
        
        print(f"Data Acquisition {stock_history_counter} | Stock Symbol : {s}")
        
        stock_history_counter = stock_history_counter+ 1
    
    except:
        print("Stock not found...!!")
    continue
    
# Print statement to inform when all records have been acquired.
print(f"------------&&&&---------------")
print(f"Data Acquisition Complete")
print(f"------------&&&&---------------")      



Beginning Data Acquisition!!!
------------&&&&-------------------


Data Acquisition 1 | Stock Symbol : AIG
Data Acquisition 2 | Stock Symbol : AFL
Data Acquisition 3 | Stock Symbol : ALL
Data Acquisition 4 | Stock Symbol : AON
Data Acquisition 5 | Stock Symbol : AJG
Data Acquisition 6 | Stock Symbol : CB
Data Acquisition 7 | Stock Symbol : CINF
Data Acquisition 8 | Stock Symbol : RE
Data Acquisition 9 | Stock Symbol : GL
Data Acquisition 10 | Stock Symbol : HIG
Data Acquisition 11 | Stock Symbol : MMC
Data Acquisition 12 | Stock Symbol : MET
Data Acquisition 13 | Stock Symbol : PFG
Data Acquisition 14 | Stock Symbol : PGR
Data Acquisition 15 | Stock Symbol : PRU
Data Acquisition 16 | Stock Symbol : TRV
Data Acquisition 17 | Stock Symbol : UNM
Data Acquisition 18 | Stock Symbol : WLTW
Data Acquisition 19 | Stock Symbol : LNC
Data Acquisition 20 | Stock Symbol : L

------------&&&&---------------
Data Acquisition Complete
------------&&&&---------------


## View Output Data & Create Data Frame

In [9]:
# print(json.dumps(prices_list, indent =4))
# print(json.dumps(event_list, indent =4))
print(prices_list)
print(event_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




[
    {
        "amount": 0.32,
        "date": 1560432600,
        "type": "DIVIDEND",
        "data": 0.32,
        "symbol": "AIG"
    },
    {
        "amount": 0.32,
        "date": 1552570200,
        "type": "DIVIDEND",
        "data": 0.32,
        "symbol": "AIG"
    },
    {
        "amount": 0.32,
        "date": 1544538600,
        "type": "DIVIDEND",
        "data": 0.32,
        "symbol": "AIG"
    },
    {
        "amount": 0.32,
        "date": 1536931800,
        "type": "DIVIDEND",
        "data": 0.32,
        "symbol": "AIG"
    },
    {
        "amount": 0.32,
        "date": 1528896600,
        "type": "DIVIDEND",
        "data": 0.32,
        "symbol": "AIG"
    },
    {
        "amount": 0.32,
        "date": 1521034200,
        "type": "DIVIDEND",
        "data": 0.32,
        "symbol": "AIG"
    },
    {
        "amount": 0.32,
        "date": 1512657000,
        "type": "DIVIDEND",
        "data": 0.32,
        "symbol": "AIG"
    },
    {
        "amount": 

]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




[{'amount': 0.32, 'date': 1560432600, 'type': 'DIVIDEND', 'data': 0.32, 'symbol': 'AIG'}, {'amount': 0.32, 'date': 1552570200, 'type': 'DIVIDEND', 'data': 0.32, 'symbol': 'AIG'}, {'amount': 0.32, 'date': 1544538600, 'type': 'DIVIDEND', 'data': 0.32, 'symbol': 'AIG'}, {'amount': 0.32, 'date': 1536931800, 'type': 'DIVIDEND', 'data': 0.32, 'symbol': 'AIG'}, {'amount': 0.32, 'date': 1528896600, 'type': 'DIVIDEND', 'data': 0.32, 'symbol': 'AIG'}, {'amount': 0.32, 'date': 1521034200, 'type': 'DIVIDEND', 'data': 0.32, 'symbol': 'AIG'}, {'amount': 0.32, 'date': 1512657000, 'type': 'DIVIDEND', 'data': 0.32, 'symbol': 'AIG'}, {'amount': 0.32, 'date': 1505395800, 'type': 'DIVIDEND', 'data': 0.32, 'symbol': 'AIG'}, {'amount': 0.32, 'date': 1497274200, 'type': 'DIVIDEND', 'data': 0.32, 'symbol': 'AIG'}, {'amount': 0.32, 'date': 1489411800, 'type': 'DIVIDEND', 'data': 0.32, 'symbol': 'AIG'}, {'amount': 0.32, 'date': 1481034600, 'type': 'DIVIDEND', 'data': 0.32, 'symbol': 'AIG'}, {'amount': 0.32, 'd

In [10]:
# Convert to Data Frame
price_history = pd.DataFrame(prices_list)
dividend_history = pd.DataFrame(event_list)

## Save data to CSV & JSON

In [11]:
## Save data to CSV and JSON

price_history.to_csv("../stocks_data/raw_price_history.csv", index=False, encoding='utf8')
dividend_history.to_csv("../stocks_data/raw_dividend_history.csv", index=False, encoding='utf8')

price_history.to_json("../stocks_data/raw_price_history.json", orient='columns')
dividend_history.to_json("../stocks_data/raw_dividend_history.json", orient='columns')


# Extract News Stock Data - Financial Services - Insurance

## Daily News Stocks - Financial Services - Insurance

In [12]:
# Yahoo Finance API - News Data

news_url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/get-news"


#symbols_insurance = ['AIG','AFL','ALL'] # testing list
# Back up variable -- symbols_insurance = ['AIG','AFL','ALL','AON','AJG','CB','CINF','RE','GL','HIG','MMC','MET','PFG','PGR','PRU','TRV','UNM','WLTW','LNC','L']


headers = {
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
    'x-rapidapi-key': "09ec0160admsh2d5563a7feaa3efp1e2db5jsn5787845ed7fc"
    }

stocks_news = []

stock_news_counter = 1

#Print statement to inform when data acquisition start
print(f"Beginning Data Acquisition!!!")
print(f"------------&&&&-------------------")

for news in tqdm(symbols_insurance):

    querystring_news = {"region":"US","category":news}

    try: 
        response_news = requests.get(news_url,headers = headers, params= querystring_news).json()
        
        stocks_news.append(response_news)
           
        print(f"Data Acquisition {stock_news_counter} | Stock Symbol : {news}")
        
        stock_news_counter = stock_news_counter + 1
    
    except:
        print("Stock not found...!!")
    continue
    
# Print statement to inform when all records have been acquired.
print(f"------------&&&&---------------")
print(f"Data Acquisition Complete")
print(f"------------&&&&---------------")    


Beginning Data Acquisition!!!
------------&&&&-------------------


Data Acquisition 1 | Stock Symbol : AIG
Data Acquisition 2 | Stock Symbol : AFL
Data Acquisition 3 | Stock Symbol : ALL
Data Acquisition 4 | Stock Symbol : AON
Data Acquisition 5 | Stock Symbol : AJG
Data Acquisition 6 | Stock Symbol : CB
Data Acquisition 7 | Stock Symbol : CINF
Data Acquisition 8 | Stock Symbol : RE
Data Acquisition 9 | Stock Symbol : GL
Data Acquisition 10 | Stock Symbol : HIG
Data Acquisition 11 | Stock Symbol : MMC
Data Acquisition 12 | Stock Symbol : MET
Data Acquisition 13 | Stock Symbol : PFG
Data Acquisition 14 | Stock Symbol : PGR
Data Acquisition 15 | Stock Symbol : PRU
Data Acquisition 16 | Stock Symbol : TRV
Data Acquisition 17 | Stock Symbol : UNM
Data Acquisition 18 | Stock Symbol : WLTW
Data Acquisition 19 | Stock Symbol : LNC
Data Acquisition 20 | Stock Symbol : L

------------&&&&---------------
Data Acquisition Complete
------------&&&&---------------


## Print output data and check length of extracted information 

In [13]:
# # Print output data and check length of extracted information 
# print(json.dumps(stocks_news, indent =4))
len(stocks_news)

20

## Loop to obtain only the key data

In [14]:
# First loop to get to the desire results
stocks_news_list = []

for i in range(len(stocks_news)):
    for j in range(len(stocks_news[i]['items']['result'])):
        stocks_news_list.append(stocks_news[i]['items']['result'][j])
        #ticker_list.append(stocks_news[i]['items']['result'][j]['entities'])
        
stocks_news_list

[{'uuid': '887a746b-c074-36a9-bd2a-f4cfbaa68870',
  'title': 'Why Is American International Group (AIG) Down 3.8% Since Last Earnings Report?',
  'link': 'https://finance.yahoo.com/news/why-american-international-group-aig-133101699.html',
  'summary': "American International Group (AIG) reported earnings 30 days ago. What's next for the stock? We take a look at earnings estimates for some clues.",
  'publisher': 'Zacks',
  'author': 'Zacks Equity Research',
  'ignore_main_image': False,
  'type': 'story',
  'published_at': 1567776661,
  'main_image': {'original_height': 741,
   'original_width': 900,
   'original_url': 'https://s.yimg.com/lo/api/res/1.2/rcafUCTxnploPCVZwyOuIw--/YXBwaWQ9eXZpZGVvZmVlZHM7dz05MDA7aD03NDE-/https://media.zenfs.com/en-us/zacks.com/7853de8d69faeccb7d1ad6ebba40b8a2',
   'resolutions': [{'tag': 'original',
     'width': 900,
     'url': 'https://s.yimg.com/lo/api/res/1.2/rcafUCTxnploPCVZwyOuIw--/YXBwaWQ9eXZpZGVvZmVlZHM7dz05MDA7aD03NDE-/https://media.zenfs.com/e

## Looping, append and create a Data Frame

In [15]:
# Creating a news data frame from the json output
# https://python-forum.io/Thread-build-pandas-dataframe-from-a-for-loop

# Data Frame
stocks_news_df = pd.DataFrame({"title": [], "source_link": [], "summary": [
], "publisher": [], "date": [], "symbol": [], 'name': []})

for news in range(len(stocks_news_list)):
        news_data = stocks_news_list[news]
        stocks_news_df = stocks_news_df.append({'title': news_data['title'], 'source_link': news_data['link'],\
                                                'summary': news_data['summary'],'publisher': news_data['publisher'],\
                                                'date': news_data['published_at'], 'symbol': news_data['entities'][0]['term'],\
                                                "name": news_data['entities'][0]['label'] }, ignore_index=True)



## Visualize all output data

In [16]:
stocks_news_df

,title,source_link,summary,publisher,date,symbol,name
0,Why Is American International Group (AIG) Down...,https://finance.yahoo.com/news/why-american-in...,American International Group (AIG) reported ea...,Zacks,1.567777e+09,TICKER:AIG,"American International Group, I"
1,Reinsurers benefit from calm amid the storm,http://www.ft.com/cms/s/34bda2a2-cee1-11e9-b01...,As the $600bn reinsurance industry’s top brass...,Financial Times,1.567652e+09,TICKER:AIG,"American International Group, I"
2,'Fast Money ' Traders Share Their Thoughts On ...,https://finance.yahoo.com/news/fast-money-trad...,"Stephanie Link said on CNBC's ""Fast Money Half...",Benzinga,1.567595e+09,TICKER:MPC,Marathon Petroleum Corporation
3,AIG Names Duncan Ellis Head of Retail Property...,https://finance.yahoo.com/news/aig-names-dunca...,"American International Group, Inc. (AIG) today...",Business Wire,1.567543e+09,TICKER:AIG,"American International Group, I"
4,What Makes Allstate (ALL) an Attractive Invest...,https://finance.yahoo.com/news/makes-allstate-...,Allstate's (ALL) well-performing Property and ...,Zacks,1.567429e+09,TICKER:AIG,"American International Group, I"
...,...,...,...,...,...,...,...
187,Key Takeaways From Loews' 2nd-Quarter Results,https://finance.yahoo.com/news/key-takeaways-l...,"Company’s earnings, revenue grew year over yea...",GuruFocus.com,1.565122e+09,TICKER:L,Loews Corporation
188,Loews Corporation Announces Quarterly Dividend...,https://finance.yahoo.com/news/loews-corporati...,"NEW YORK , Aug. 6, 2019 /PRNewswire/ -- Loews ...",PR Newswire,1.565108e+09,TICKER:CNA,CNA Financial Corporation
189,Loews (L) Q2 2019 Earnings Call Transcript,https://finance.yahoo.com/news/loews-l-q2-2019...,"L earnings call for the period ending June 30,...",Motley Fool,1.565055e+09,TICKER:L,Loews Corporation
190,US Indexes Close Lower With China Trade Worries,https://finance.yahoo.com/news/us-indexes-clos...,S&P; 500 down 2.98% Continue reading...,GuruFocus.com,1.565042e+09,TICKER:AAPL,Apple Inc.


## Save data to CSV & JSON

In [17]:
# Savings to CSV and JSON files

stocks_news_df.to_csv("../stocks_data/raw_stock_news_data.csv", index=False, encoding='utf8')

stocks_news_df.to_json("../stocks_data/raw_stock_news_data.json", orient='columns')

# Extracting Financial Statements

## Financial Statements - Last 4 Quarters

In [26]:
# Yahoo Finance API for Financials Statements

financials_url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-financials"

# symbols_insurance = ['AIG','AFL','ALL'] # testing list
# Back up variable -- symbols_insurance = ['AIG','AFL','ALL','AON','AJG','CB','CINF','RE','GL','HIG','MMC','MET','PFG','PGR','PRU','TRV','UNM','WLTW','LNC','L']


headers = {
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
    'x-rapidapi-key': "09ec0160admsh2d5563a7feaa3efp1e2db5jsn5787845ed7fc"
    }

balance_sheets = []
income_sheets = []
cashflow_sheets = []
#stocks_financials = []

stock_financials_counter = 1

#Print statement to inform when data acquisition start
print(f"Beginning Data Acquisition!!!")
print(f"------------&&&&-------------------")

for financials in tqdm(symbols_insurance):
    
    querystring_financials = {"symbol":financials}
    
    try: 
        response_financials = requests.get(financials_url,headers = headers, params=querystring_financials).json()
        
        #stocks_financials.append(response_financials)
        balance = response_financials['balanceSheetHistoryQuarterly']["balanceSheetStatements"]
        income = response_financials["incomeStatementHistoryQuarterly"]["incomeStatementHistory"]
        cashflow = response_financials['cashflowStatementHistoryQuarterly']["cashflowStatements"]
        
 
        for bal in balance:
            bal['symbol'] = financials
            balance_sheets.append(bal)
        
        for inc in income:
            inc['symbol'] = financials
            income_sheets.append(inc)
            
        for cash in cashflow:
            cash['symbol'] = financials
            cashflow_sheets.append(cash)
        

        print(f"Data Acquisition {stock_financials_counter} | Stock Symbol : {financials}")
        
        stock_financials_counter = stock_financials_counter + 1
    
    except:
        print("Stock not found...!!")
    continue
    
# Print statement to inform when all records have been acquired.
print(f"------------&&&&---------------")
print(f"Data Acquisition Complete")
print(f"------------&&&&---------------")  

Beginning Data Acquisition!!!
------------&&&&-------------------


Data Acquisition 1 | Stock Symbol : AIG
Data Acquisition 2 | Stock Symbol : AFL
Data Acquisition 3 | Stock Symbol : ALL
Data Acquisition 4 | Stock Symbol : AON
Data Acquisition 5 | Stock Symbol : AJG
Data Acquisition 6 | Stock Symbol : CB
Data Acquisition 7 | Stock Symbol : CINF
Data Acquisition 8 | Stock Symbol : RE
Data Acquisition 9 | Stock Symbol : GL
Data Acquisition 10 | Stock Symbol : HIG
Data Acquisition 11 | Stock Symbol : MMC
Data Acquisition 12 | Stock Symbol : MET
Data Acquisition 13 | Stock Symbol : PFG
Data Acquisition 14 | Stock Symbol : PGR
Data Acquisition 15 | Stock Symbol : PRU
Data Acquisition 16 | Stock Symbol : TRV
Data Acquisition 17 | Stock Symbol : UNM
Data Acquisition 18 | Stock Symbol : WLTW
Data Acquisition 19 | Stock Symbol : LNC
Data Acquisition 20 | Stock Symbol : L

------------&&&&---------------
Data Acquisition Complete
------------&&&&---------------


## Display output data

In [27]:
# Display output data
balance_sheets
print(len(balance_sheets))
print(len(cashflow_sheets))
print(len(income_sheets))

80
80
80


## Convert dict to DataFrame list

In [28]:
balance_list = []
for bbb in range(len(balance_sheets)):
    balancex = pd.DataFrame.from_dict(balance_sheets[bbb],orient='columns')
    balance_list.append(balancex)
    
income_list = []
for iii in range(len(income_sheets)):
    incomex = pd.DataFrame.from_dict(income_sheets[iii],orient='columns')
    income_list.append(incomex)
    
cashflow_list = []
for ccc in range(len(cashflow_sheets)):
    cashx = pd.DataFrame.from_dict(cashflow_sheets[ccc],orient='columns')
    cashflow_list.append(cashx)

## Concatenante Data for each financial Statements

In [29]:
# Cancatenate all data per quater for balance sheets, cashflow and income statements
# Help = https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop

balance_sheets_df = pd.concat(balance_list, sort = False)

income_sheets_df = pd.concat(income_list, sort = False)

cashflow_sheets_df = pd.concat(cashflow_list, sort = False)

## Display Data Frame Results

In [30]:
# Display data results

balance_sheets_df
income_sheets_df
cashflow_sheets_df


,maxAge,endDate,netIncome,changeToLiabilities,symbol,investments,totalCashflowsFromInvestingActivities,netBorrowings,totalCashFromFinancingActivities,changeToOperatingActivities,...,totalCashFromOperatingActivities,depreciation,otherCashflowsFromInvestingActivities,dividendsPaid,changeToAccountReceivables,otherCashflowsFromFinancingActivities,changeToNetincome,repurchaseOfStock,issuanceOfStock,capitalExpenditures
raw,1,1561852800,1109000000,0,AIG,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fmt,1,2019-06-30,1.11B,None,AIG,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
longFmt,1,NaN,"1,109,000,000",0,AIG,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
raw,1,1553990400,654000000,0,AIG,-1353000000,-1979000000,860000000,2764000000,-4073000000,...,-976000000,1299000000,17000000,-278000000,315000000,1697000000,662000000,NaN,NaN,NaN
fmt,1,2019-03-31,654M,None,AIG,-1.35B,-1.98B,860M,2.76B,-4.07B,...,-976M,1.3B,17M,-278M,315M,1.7B,662M,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fmt,1,2018-12-31,-165M,None,L,-387M,-727M,-91M,-260M,432M,...,827M,863M,-56M,-20M,-149M,-12M,-357M,-137M,NaN,-264M
longFmt,1,NaN,"-165,000,000",0,L,"-387,000,000","-727,000,000","-91,000,000","-260,000,000","432,000,000",...,"827,000,000","863,000,000","-56,000,000","-20,000,000","-149,000,000","-12,000,000","-357,000,000","-137,000,000",NaN,"-264,000,000"
raw,1,1538265600,278000000,0,L,695000000,520000000,-15000000,-1647000000,-414000000,...,1244000000,16000000,-28000000,-20000000,525000000,-1522000000,622000000,-90000000,NaN,-251000000
fmt,1,2018-09-30,278M,None,L,695M,520M,-15M,-1.65B,-414M,...,1.24B,16M,-28M,-20M,525M,-1.52B,622M,-90M,NaN,-251M


## Save data to CSV

In [31]:
# Save dataframe to CSV

balance_sheets_df.to_csv("../stocks_data/raw_balance_sheets_data.csv", index=False, encoding='utf8')
cashflow_sheets_df.to_csv("../stocks_data/raw_cashflow_sheets_data.csv", index=False, encoding='utf8')
income_sheets_df.to_csv("../stocks_data/raw_income_sheets_data.csv", index=False, encoding='utf8')


# Move to Transformation 